In [ ]:
!pip install numpy scipy gensim faiss-gpu fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 23.4 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296184 sha256=2d36191ae08f864744c9735516bac5f92039676bdaca0856b3e843857ba2490b
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3998, done.
remote: Counting objects: 100% (1057/1057), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 3998 (delta 922), reused 884 (delta 856), pack-reused 2941 (from 1)
Receiving objects: 100% (3998/3998), 8.30 MiB | 13.42 MiB/s, done.
Resolving deltas: 100% (2529/2529), done.


In [ ]:
!git clone https://github.com/facebookresearch/MUSE.git

Cloning into 'MUSE'...
remote: Enumerating objects: 239, done.
remote: Total 239 (delta 0), reused 0 (delta 0), pack-reused 239 (from 1)
Receiving objects: 100% (239/239), 215.77 KiB | 894.00 KiB/s, done.
Resolving deltas: 100% (136/136), done.


In [ ]:
!wget https://dl.fbaipublicfiles.com/arrival/dictionaries.tar.gz

--2024-10-07 17:01:19--  https://dl.fbaipublicfiles.com/arrival/dictionaries.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.78, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62097248 (59M) [application/gzip]
Saving to: ‘dictionaries.tar.gz’

dictionaries.tar.gz 100%[===================>]  59.22M  35.6MB/s    in 1.7s    

2024-10-07 17:01:21 (35.6 MB/s) - ‘dictionaries.tar.gz’ saved [62097248/62097248]



In [ ]:
!cp dictionaries.tar.gz /content/MUSE/data/dictionaries.tar.gz

In [ ]:
!tar -xvzf /content/MUSE/data/dictionaries.tar.gz  -C /content/MUSE/data

In [ ]:
!./fastText/download_model.py en

 (100.00%) [==================================================>]


In [ ]:
!./fastText/download_model.py hi

 (100.00%) [==================================================>]


In [ ]:
import os
import shutil
import gzip
# Function to extract a .gz file
def extract_gz_file(gz_file_path, output_path):
    if not os.path.exists(output_path):
        print(f"Extracting {gz_file_path}...")
        with gzip.open(gz_file_path, 'rb') as f_in:
            with open(output_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Extracted to {output_path}")
    else:
        print(f"{output_path} already exists, skipping extraction.")

In [ ]:
# Define paths for the .gz files and their extracted versions
gz_file_path_en = "cc.en.300.bin.gz"
output_path_en = "cc.en.300.bin"

gz_file_path_hi = "cc.hi.300.bin.gz"
output_path_hi = "cc.hi.300.bin"

# Extract the .gz files
extract_gz_file(gz_file_path_en, output_path_en)
extract_gz_file(gz_file_path_hi, output_path_hi)

cc.en.300.bin already exists, skipping extraction.
cc.hi.300.bin already exists, skipping extraction.


In [ ]:
import gensim

# Load pre-trained FastText models
en_model = gensim.models.fasttext.load_facebook_vectors('cc.en.300.bin')
hi_model = gensim.models.fasttext.load_facebook_vectors('cc.hi.300.bin')


In [ ]:
import numpy as np
def limit_vocab(model, top_n=100000):
    # Get top N most frequent words
    vocab = list(model.key_to_index.keys())[:top_n]
    vectors = model[vocab]
    return vocab, vectors

en_vocab, en_vectors = limit_vocab(en_model, 100000)
hi_vocab, hi_vectors = limit_vocab(hi_model, 100000)
# Normalize the embeddings
en_vectors = en_vectors / np.linalg.norm(en_vectors, axis=1, keepdims=True)
hi_vectors = hi_vectors / np.linalg.norm(hi_vectors, axis=1, keepdims=True)


In [ ]:
def load_bilingual_lexicon(path):
    word_pairs = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            en_word, hi_word = line.strip().split()
            word_pairs.append((en_word, hi_word))
    return word_pairs

# Replace with the path to your MUSE dictionary file
bilingual_lexicon = load_bilingual_lexicon('MUSE/data/dictionaries/en-hi.txt')


In [ ]:
import numpy as np
from scipy.linalg import orthogonal_procrustes

# Extract embeddings for aligned words
def get_aligned_embeddings(lexicon, en_vocab, en_vectors, hi_vocab, hi_vectors):
    en_embed, hi_embed = [], []
    en_index = {word: idx for idx, word in enumerate(en_vocab)}
    hi_index = {word: idx for idx, word in enumerate(hi_vocab)}

    for en_word, hi_word in lexicon:
        if en_word in en_index and hi_word in hi_index:
            en_embed.append(en_vectors[en_index[en_word]])
            hi_embed.append(hi_vectors[hi_index[hi_word]])

    return np.array(en_embed), np.array(hi_embed)

en_aligned, hi_aligned = get_aligned_embeddings(bilingual_lexicon, en_vocab, en_vectors, hi_vocab, hi_vectors)

# Procrustes alignment to find the best orthogonal mapping
def procrustes(X, Y):
    R, _ = orthogonal_procrustes(X, Y)
    return R


# Get the rotation matrix
R = procrustes(en_aligned, hi_aligned)

# Apply the transformation to align English vectors to the Hindi space
en_aligned_vectors = en_vectors @ R


In [ ]:
import faiss

def get_translation_accuracy(source_vectors, target_vectors, source_vocab, target_vocab, bilingual_lexicon, top_k=5):
    # Create a FAISS index and add the target vectors to it for fast nearest neighbor search
    index = faiss.IndexFlatL2(target_vectors.shape[1])
    index.add(target_vectors.astype('float32'))

    # Perform a search for the nearest neighbors
    _, neighbors = index.search(source_vectors.astype('float32'), top_k)

    # Initialize dictionary to keep track of top-k accuracy counts
    top_k_accuracy = {k: 0 for k in range(1, top_k + 1)}

    # Iterate over each word in the source vocabulary and its corresponding translations
    for i, (en_word, hi_word) in enumerate(bilingual_lexicon):
        # Find the nearest neighbors (indices) in the target vocabulary
        nearest_indices = neighbors[i]

        # Get the corresponding words for the nearest neighbors
        nearest_words = [target_vocab[idx] for idx in nearest_indices]

        # Check if the correct translation is among the top-k nearest neighbors
        for k in range(1, top_k + 1):
            if hi_word in nearest_words[:k]:
                top_k_accuracy[k] += 1

    # Calculate and print the Precision@k for each k
    for k in top_k_accuracy:
        top_k_accuracy[k] /= len(bilingual_lexicon)
        print(f'Precision@{k}: {top_k_accuracy[k] * 100:.2f}%')

# Use the updated function with the aligned vectors and bilingual lexicon
get_translation_accuracy(en_aligned_vectors, hi_vectors, en_vocab, hi_vocab, bilingual_lexicon)


Precision@1: 0.00%
Precision@2: 0.01%
Precision@3: 0.01%
Precision@4: 0.02%
Precision@5: 0.02%


In [ ]:
%%time
# Use the first 20,000 word pairs if available in the bilingual lexicon
lexicon_subset = bilingual_lexicon[:20000]
en_aligned, hi_aligned = get_aligned_embeddings(lexicon_subset, en_vocab, en_vectors, hi_vocab, hi_vectors)
R = procrustes(en_aligned, hi_aligned)
en_aligned_vectors = en_vectors @ R

print(f'\nResults for lexicon size 20,000:')
get_translation_accuracy(en_aligned_vectors, hi_vectors, en_vocab, hi_vocab, lexicon_subset)



Results for lexicon size 20,000:
Precision@1: 0.03%
Precision@2: 0.03%
Precision@3: 0.03%
Precision@4: 0.04%
Precision@5: 0.06%
CPU times: user 5min 4s, sys: 2.59 s, total: 5min 6s
Wall time: 1min 33s


In [ ]:
def iterative_procrustes(source_embeddings, target_embeddings, initial_R, source_vocab, target_vocab, iterations=5):
    R = initial_R
    for i in range(iterations):
        # Align the source embeddings
        source_aligned = source_embeddings @ R

        # Find new word pairs using nearest neighbors
        index = faiss.IndexFlatL2(target_embeddings.shape[1])
        index.add(target_embeddings.astype('float32'))
        _, neighbors = index.search(source_aligned.astype('float32'), 1)

        # Create new pairs based on the nearest neighbors
        new_pairs = [(source_vocab[i], target_vocab[neighbors[i][0]]) for i in range(len(source_vocab))]

        # Use new pairs to refine R
        new_src, new_tgt = get_aligned_embeddings(new_pairs, source_vocab, source_embeddings, target_vocab, target_embeddings)

        # Re-compute the Procrustes transformation matrix
        R = procrustes(new_src, new_tgt)

    return R


In [ ]:
%%time

# Assuming source_vocab and target_vocab are defined earlier in your code
initial_R = procrustes(en_aligned, hi_aligned)

# Run iterative refinement with vocabularies
refined_R = iterative_procrustes(en_vectors, hi_vectors, initial_R, en_vocab, hi_vocab, iterations=5)

# Apply the refined transformation
en_aligned_vectors = en_vectors @ refined_R

# Re-evaluate accuracy
print(f'\nResults for refined Procrustes:')
get_translation_accuracy(en_aligned_vectors, hi_vectors, en_vocab, hi_vocab, lexicon_subset)



Results for refined Procrustes:
Precision@1: 0.00%
Precision@2: 0.03%
Precision@3: 0.03%
Precision@4: 0.03%
Precision@5: 0.03%
CPU times: user 30min 16s, sys: 16.5 s, total: 30min 32s
Wall time: 9min 23s


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compute_csls(source_vectors, target_vectors, k=5):
    # Compute cosine similarities
    sim_matrix = cosine_similarity(source_vectors, target_vectors)

    # Get the top k nearest neighbors
    knn_source = np.argsort(sim_matrix, axis=1)[:, -k:]
    knn_target = np.argsort(sim_matrix, axis=0)[-k:, :]

    # Compute CSLS scores
    csls_scores = np.zeros(sim_matrix.shape)

    for i in range(sim_matrix.shape[0]):
        for j in range(sim_matrix.shape[1]):
            sim_xy = sim_matrix[i, j]
            avg_neigh_source = np.mean(sim_matrix[i, knn_target[:, j]])
            avg_neigh_target = np.mean(sim_matrix[knn_source[i], j])
            csls_scores[i, j] = 2 * sim_xy - avg_neigh_source - avg_neigh_target

    return csls_scores


In [ ]:
def evaluate_translation(csls_scores, bilingual_lexicon, top_k=5):
    top_k_accuracy = {k: 0 for k in range(1, top_k + 1)}

    for i, (source_word, target_word) in enumerate(bilingual_lexicon):
        best_indices = np.argsort(-csls_scores[i])[:top_k]
        best_words = [target_vocab[idx] for idx in best_indices]

        for k in range(1, top_k + 1):
            if target_word in best_words[:k]:
                top_k_accuracy[k] += 1

    for k in top_k_accuracy:
        top_k_accuracy[k] /= len(bilingual_lexicon)
        print(f'CSLS Precision@{k}: {top_k_accuracy[k] * 100:.2f}%')

# Example of usage
csls_scores = compute_csls(en_vectors, hi_vectors)
evaluate_translation(csls_scores, bilingual_lexicon)
